# Problem Formulation

Consider a *loss function* $g: \mathbb{R}^d \to \mathbb{R}$. We say that an event $x \in \mathbb{R}^d$ is a *failure* if $g(x)$ is greater than the *critical threshold* $\alpha$. 

The set of all $x \in \mathbb{R}^d$ for which $X$ is a *failure* is known as the *failure domain*
$$F = \{ X \in \mathbb{R}^d : g(X) > \alpha \} \subset \mathbb{R}^d.$$

If $X$ is sampled from a distribution $\pi$, then the *failure probability* is

$$p_F := \Pr(X \in F) =  \int_F \pi(x) dx $$

In practical applications, it is usually the case that
- $\pi(x) = \mathcal{N}(0,1)$, 
- $g(x)$ is expensive to compute,
- $p_F << 1$ ($p_F \sim 10^{-2}, 10^{-9}$), and
- $d >> 1$ (up to $10^3$).

We want to minimize the computations of $g$ necessary to determine $p_F$ accurately for a given $\alpha$. Since $p_F$ is very small, we will use the *coefficient of variation* 
$$\delta = \frac{\sigma[\hat{p}_F]}{\mathbb{E}[\hat{p}_F]}$$
as our measure of accuracy.

In [1]:
%use krangl, lets-plot

## Sandbox Loss Functions

We will use the following $g$, which are easy to compute, in our analysis:
- (*Linear*) $g(x) = \sum_{i=1}^d a_i x_i$,
- (*Quadratic*) $g(x) = \sum_{i=1}^d a_i x_i + \sum_{i,j = 1}^d b_{ij} x_i x_j$, and
- (*Brownian*) $g(x) = \max_k B_{t_k}$ where 
    - $0 = t_0 < \cdots < t_d = T$ for some interval $T$, and
    - $B_{t_k} = \sum_{i=1}^d \sqrt{t_i - t_{i-1}} X_i$.
    
Let's look at $\delta$ as a function of $\log p_F$ for Monte Carlo. We'll use the Brownian loss function because it is the fastest to run.

*Note:* The Brownian loss function is the fastest to run because it doesn't have to sample itself to determine the best $\alpha$. In the Brownian case, $p_F$ and $\alpha$ are related by
$$p_F = 2\big (1 - \Phi(\tfrac{\alpha}{\sqrt{T}}) \big)$$
where $\Phi$ is the standard normal CDF.

In [14]:
var example = DataFrame.readCSV("test.csv").filter { it["methodType"] eq "MC" }
example.head()

methodType,gType,gCalls,dim,pF,pFHat
MC,Brownian,100000,10,0.1,0.06782932170678294
MC,Brownian,100000,10,0.1,0.07054929450705492
MC,Brownian,100000,10,0.1,0.07181928180718193
MC,Brownian,100000,10,0.1,0.06835931640683593
MC,Brownian,100000,10,0.1,0.056859431405685945


Let's plot $\log p_F$ against $\delta$ for the Monte Carlo in the Brownian case using this data.

In [16]:
val pFList = example["pF"]
    .toDoubles()
    .associateBy { it }
    .keys
    .toList()
    .sortedBy { it }
val deltaList = pFList
    .map { pF ->
        example
            .filter { it["pF"] eq pF!! }["pFHat"]
            .sd()!! / pF!!
    }
val summary = mapOf<String, List<*>>(
    "logpF" to pFList.map { log10(it!!) },
    "delta" to deltaList
)

val p = letsPlot(summary) { 
        x="logpF"
        y="delta"
    }

p + geomLine()